In [1]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1,
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

# llm.invoke(prompt)

In [9]:
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

def plus(a, b):
    return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool take two arguments, both  should be numbers.",
        ),
    ],
)

prompt = "Tell me about the sum result of the cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

result = agent.invoke(prompt)

print(result)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the sum of all these costs.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```

Observation: 1280.26
Thought:I have calculated the sum of the first two costs correctly. Now, I will continue to calculate the sum of all the remaining costs.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}
```
Observation: 2001.46
Thought:I have calculated the sum of the first three costs correctly. Now, I will continue to calculate the sum of all the remaining costs.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 2001.46, "b": 1940.29}
}
```
Observation: 3941.75
Thought:I have calculated the sum of all the costs correctly so far. Now, I will continue to calculate the sum of the remaining costs.
Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 3941.75, "b": 573.63}
}
```
Observation: 4515.38


In [11]:
from typing import Type
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel,Field

class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The First number")
    b: float = Field(description="The Second number")

class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers
    Only receives two arguments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool()
    ],
)

prompt = "Tell me about the sum result of the cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

result = agent.invoke(prompt)

print(result)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 355.39, 'b': 924.87}`


1280.26
Invoking: `CalculatorTool` with `{'a': 1280.26, 'b': 721.2}`


2001.46
Invoking: `CalculatorTool` with `{'a': 2001.46, 'b': 1940.29}`


3941.75
Invoking: `CalculatorTool` with `{'a': 3941.75, 'b': 573.63}`


4515.38
Invoking: `CalculatorTool` with `{'a': 4515.38, 'b': 65.72}`


4581.1
Invoking: `CalculatorTool` with `{'a': 4581.1, 'b': 35.0}`


4616.1
Invoking: `CalculatorTool` with `{'a': 4616.1, 'b': 552.0}`


5168.1
Invoking: `CalculatorTool` with `{'a': 5168.1, 'b': 76.16}`


5244.26
Invoking: `CalculatorTool` with `{'a': 5244.26, 'b': 29.12}`


5273.38The sum of the costs $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.
{'input': 'Tell me about the sum result of the cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12', 'output': 'The sum of the c

In [2]:
from typing import Type
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel,Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for.Example query: Stock Markey Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company
    It takes a query as an argument.
    
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool()
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Aloso tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`


Cloudflare's stock has recently been incredibly volatile, ranging between approximately $30 and $80. The significant price movements have been in response to news and events affecting the broader tech industry. Cloudflare operates in the highly competitive cybersecurity industry, characterized by rapid technological change, intense competition ... Find the latest Cloudflare, Inc. (NET) stock analysis from Seeking Alpha's top analysts: exclusive research and insights from bulls and bears. Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. View Cloudflare Inc Class A NET stock quote prices, financial information, real-time forecasts, and company news from CNN. Cloudflare shares were sharply higher in pre-market trading after the company delivered strong quarterly results an

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Aloso tell me what symbol does the stock have.",
 'output': "Cloudflare's stock symbol is NET. The stock has been experiencing significant volatility, with prices ranging between approximately $30 and $80. The cybersecurity industry in which Cloudflare operates is highly competitive and subject to rapid technological changes.\n\nTo analyze if Cloudflare is a potential good investment, we need to consider various factors such as the company's financial performance, growth prospects, competitive position, and market trends. It's essential to conduct a thorough analysis of the company's financial statements, revenue growth, profitability, and future outlook.\n\nAdditionally, you may want to consider factors such as industry trends, market conditions, and potential risks associated with investing in Cloudflare. It's recommended to consult with a financial advisor or conduct further